In [1]:
import numpy as np

Definições Geométricas

In [26]:
Lx = 1
Nx = 20

Ly = 1
Ny = 20

Npoints = Nx*Ny

Definições do material

In [27]:
rho = 1
cv = 1
alpha = 1

In [28]:
X = np.linspace(0, Lx, Nx, dtype = float)
Y = np.linspace(0, Ly, Ny, dtype = float)

Xv, Yv = np.meshgrid(X, Y)

X = np.reshape(Xv, Npoints)
Y = np.reshape(Yv, Npoints)

In [29]:
## IEN 
# 0  ,  nx      ,  nx + 1 : type 1
# 0  ,  nx + 1  ,  1      : type 2

IEN = [
    [0, Nx + 1, Nx    ],
    [0,      1, Nx + 1]
]

for i in range(Ny - 1):
    
    for j in range(Nx - 2):
        IEN.append([IEN[-2][k] + 1 for k in range(3)])
        IEN.append([IEN[-2][k] + 1 for k in range(3)])

    if i < Ny - 2:
        IEN.append([IEN[0][k] + Nx*(i + 1) for k in range(3)])
        IEN.append([IEN[1][k] + Nx*(i + 1) for k in range(3)])


In [30]:
#form functions for 2D
def IENfunc(K, M, IEN):
    for IENelem in IEN:
        x_list = [X[IENelem[i]] for i in range(3)]
        y_list = [Y[IENelem[i]] for i in range(3)] #find direct slicing of x and y elems

        tri_matrix = [[1, X[IENelem[i]], Y[IENelem[i]]] for i in range(3)]

        tri_area = abs(np.linalg.det(tri_matrix)/2)

        a_list = np.array([x_list[(i + 1)%3]*y_list[(i + 2)%3] - x_list[(i + 2)%3]*y_list[(i + 1)%3] for i in range(3)])/(2*tri_area)
        b_list = np.array([y_list[(i + 1)%3] - y_list[(i + 2)%3] for i in range(3)])/(2*tri_area)
        c_list = np.array([x_list[(i + 2)%3] - x_list[(i + 1)%3] for i in range(3)])/(2*tri_area)

        N_list = np.array([[a_list[i], b_list[i], c_list[i]] for i in range(3)])

        melem = tri_area/12*np.array(([2, 1, 1],
                                      [1, 2, 1],
                                      [1, 1, 2]))
        
        kxelem = (alpha/(4*tri_area))*np.array([[b_list[i]*b_list[j] for i in range(3)] for j in range(3)])
        kyelem = (alpha/(4*tri_area))*np.array([[c_list[i]*c_list[j] for i in range(3)] for j in range(3)])

        kelem = kxelem + kyelem

        for ilocal in range(3):
            iglobal = IENelem[ilocal]

            for jlocal in range(3):
                jglobal = IENelem[jlocal]

                K[iglobal,jglobal] += kelem[ilocal,jlocal]
                M[iglobal,jglobal] += melem[ilocal,jlocal]

In [31]:
K  = np.zeros((Npoints, Npoints), dtype = float) 
M  = np.zeros((Npoints, Npoints), dtype = float)

IENfunc(K, M, IEN)

print(K)
print(M)

[[130321.  -65160.5      0.  ...      0.       0.       0. ]
 [-65160.5 260642.  -65160.5 ...      0.       0.       0. ]
 [     0.  -65160.5 260642.  ...      0.       0.       0. ]
 ...
 [     0.       0.       0.  ... 260642.  -65160.5      0. ]
 [     0.       0.       0.  ... -65160.5 260642.  -65160.5]
 [     0.       0.       0.  ...      0.  -65160.5 130321. ]]
[[0.00046168 0.00011542 0.         ... 0.         0.         0.        ]
 [0.00011542 0.00069252 0.00011542 ... 0.         0.         0.        ]
 [0.         0.00011542 0.00069252 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.00069252 0.00011542 0.        ]
 [0.         0.         0.         ... 0.00011542 0.00069252 0.00011542]
 [0.         0.         0.         ... 0.         0.00011542 0.00046168]]


In [32]:
Lx = 1
Nx = 2

Ly = 1
Ny = 2

Lz = 1
Nz = 2

Npoints = Nx*Ny*Nz

In [97]:
#form functions for 3D
X = np.linspace(0, Lx, Nx)
Y = np.linspace(0, Ly, Ny)
Z = np.linspace(0, Lz, Nz)

Zv, Xv, Yv = np.meshgrid(Z, X, Y) # the Z, Y, X order is just so Xv is neatly ordered

X = np.reshape(Xv, Npoints)
Y = np.reshape(Yv, Npoints)
Z = np.reshape(Zv, Npoints)

IEN3D = [
    [0, 1, 3, 7],
    [0, 1, 4, 7],
    [1, 2, 3, 7],
    [1, 2, 6, 7],
    [1, 4, 5, 7],
    [1, 5, 6, 7],
]

In [134]:
def IEN3Dfunctet(IEN3Delem: list):
    tet_matrix = [[1, X[vertex_idx], Y[vertex_idx], Z[vertex_idx]] for vertex_idx in IEN3Delem]
    
    tet_matrix_inv = np.linalg.inv(tet_matrix)

    # tet_volume = abs(np.linalg.det(tet_matrix)/6)

    univector = lambda i : np.array([[i == j] for j in range(4)])

    N_list = np.zeros((4, 4))

    for i in range(4):
        N_list[i] = np.transpose(tet_matrix_inv@univector(i))
    
    return N_list